In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn


import sklearn
from sklearn import preprocessing
from datetime import datetime
import random
import math
import holidays
us_holidays = holidays.UnitedStates()


## Preprocess Weather Data

In [10]:
df_w = pd.read_csv('Data/weather_train.csv')

# df_w['month'] = pd.DatetimeIndex(df_w['timestamp']).month.astype(str)
# df_w['day'] = pd.DatetimeIndex(df_w['timestamp']).day

# df_w['site_id'] = df_w['site_id'].astype(str)

df_w2 = df_w.copy(deep=True)

df_w2['air_temperature'] = df_w2['air_temperature'].interpolate(method='quadratic', limit_direction='both')
df_w2['air_temperature'] = df_w2['air_temperature'].interpolate(method='nearest', limit_direction='both')

df_w2['dew_temperature'] = df_w2['dew_temperature'].interpolate(method='linear', limit_direction='both')
df_w2['dew_temperature'] = df_w2['dew_temperature'].interpolate(method='nearest', limit_direction='both')

df_w2['sea_level_pressure'] = df_w2['sea_level_pressure'].interpolate(method='quadratic', limit_direction='both')
df_w2['sea_level_pressure'] = df_w2['sea_level_pressure'].interpolate(method='nearest', limit_direction='both')
df_w2['sea_level_pressure'] = np.where((df_w2['site_id']==5), np.nan, df_w2['sea_level_pressure'])

df_w2['wind_speed'] = df_w2['wind_speed'].interpolate(method='linear', limit_direction='both')
df_w2['wind_speed'] = df_w2['wind_speed'].interpolate(method='nearest', limit_direction='both')


df_w2['wind_direction cat'] = np.where(df_w2['wind_direction'].isna(), 'Unknown',
                                  np.where(df_w2['wind_speed']==0, 'No Wind',
                                  np.where((df_w2['wind_direction']<22.5)  | (df_w2['wind_direction']>=337.5), 'East',
                                  np.where((df_w2['wind_direction']<67.5)  & (df_w2['wind_direction']>=22.5), 'Northeast',
                                  np.where((df_w2['wind_direction']<112.5) & (df_w2['wind_direction']>=67.5), 'North',
                                  np.where((df_w2['wind_direction']<157.5) & (df_w2['wind_direction']>=112.5), 'Northwest',
                                  np.where((df_w2['wind_direction']<202.5) & (df_w2['wind_direction']>=157.5), 'West',
                                  np.where((df_w2['wind_direction']<247.5) & (df_w2['wind_direction']>=202.5), 'Southwest',
                                  np.where((df_w2['wind_direction']<292.5) & (df_w2['wind_direction']>=247.5), 'South', 
                                  'Southeast')))))))))

df_w2['wind_direction cat2'] = np.where(df_w2['wind_direction'].isna(), 'Unknown',
                                  np.where(df_w2['wind_speed']==0, 'No Wind',
                                  np.where((df_w2['wind_direction']<45)  & (df_w2['wind_direction']>=0), 'East',
                                  np.where((df_w2['wind_direction']<90)  & (df_w2['wind_direction']>=45), 'Northeast',
                                  np.where((df_w2['wind_direction']<135) & (df_w2['wind_direction']>=90), 'North',
                                  np.where((df_w2['wind_direction']<180) & (df_w2['wind_direction']>=135), 'Northwest',
                                  np.where((df_w2['wind_direction']<225) & (df_w2['wind_direction']>=180), 'West',
                                  np.where((df_w2['wind_direction']<270) & (df_w2['wind_direction']>=225), 'Southwest',
                                  np.where((df_w2['wind_direction']<315) & (df_w2['wind_direction']>=270), 'South', 
                                  'Southeast')))))))))


df_w2['precipitation'] = np.where(df_w2['precip_depth_1_hr'].isna(), 'Unknown', 
                                  np.where(df_w2['precip_depth_1_hr']<=0, 'N', 'Y'))

df_w2['precipitation sign'] = np.where(df_w2['precip_depth_1_hr'].isna(), 'Unknown',
                                   np.where(df_w2['precip_depth_1_hr']<0, 'Negative',
                                   np.where(df_w2['precip_depth_1_hr']==0, 'Zero', 'Positive')))

# df_w2['wind_speed_log'] = np.log(df_w2['wind_speed']+1)


In [12]:
df_w2.columns

Index(['site_id', 'timestamp', 'air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed', 'wind_direction cat',
       'wind_direction cat2', 'precipitation', 'precipitation sign'],
      dtype='object')

In [19]:
#####
# weather shift
shift_hour = 1
######

cols = ['air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed', 'wind_direction cat',
       'wind_direction cat2', 'precipitation', 'precipitation sign']

for col in cols:
    df_w2[col+'_lag'] = df_w2[col].shift(shift_hour)
    df_w2[col+'_lag'] = df_w2[col].shift(shift_hour)


In [22]:
df_wt = pd.read_csv('Data/weather_test.csv')

# df_wt['month'] = pd.DatetimeIndex(df_wt['timestamp']).month.astype(str)
# df_wt['day'] = pd.DatetimeIndex(df_wt['timestamp']).day
# df_wt['site_id'] = df_wt['site_id'].astype(str)

df_wt2 = df_wt.copy(deep=True)

df_wt2['air_temperature'] = df_wt2['air_temperature'].interpolate(method='quadratic', limit_direction='both')
df_wt2['air_temperature'] = df_wt2['air_temperature'].interpolate(method='nearest', limit_direction='both')

df_wt2['dew_temperature'] = df_wt2['dew_temperature'].interpolate(method='linear', limit_direction='both')
df_wt2['dew_temperature'] = df_wt2['dew_temperature'].interpolate(method='nearest', limit_direction='both')

df_wt2['sea_level_pressure'] = df_wt2['sea_level_pressure'].interpolate(method='quadratic', limit_direction='both')
df_wt2['sea_level_pressure'] = df_wt2['sea_level_pressure'].interpolate(method='nearest', limit_direction='both')
df_wt2['sea_level_pressure'] = np.where((df_wt2['site_id']==5), np.nan, df_wt2['sea_level_pressure'])

df_wt2['wind_speed'] = df_wt2['wind_speed'].interpolate(method='linear', limit_direction='both')
df_wt2['wind_speed'] = df_wt2['wind_speed'].interpolate(method='nearest', limit_direction='both')


df_wt2['wind_direction cat'] = np.where(df_wt2['wind_direction'].isna(), 'Unknown',
                                  np.where(df_wt2['wind_speed']==0, 'No Wind',
                                  np.where((df_wt2['wind_direction']<22.5)  | (df_wt2['wind_direction']>=337.5), 'East',
                                  np.where((df_wt2['wind_direction']<67.5)  & (df_wt2['wind_direction']>=22.5), 'Northeast',
                                  np.where((df_wt2['wind_direction']<112.5) & (df_wt2['wind_direction']>=67.5), 'North',
                                  np.where((df_wt2['wind_direction']<157.5) & (df_wt2['wind_direction']>=112.5), 'Northwest',
                                  np.where((df_wt2['wind_direction']<202.5) & (df_wt2['wind_direction']>=157.5), 'West',
                                  np.where((df_wt2['wind_direction']<247.5) & (df_wt2['wind_direction']>=202.5), 'Southwest',
                                  np.where((df_wt2['wind_direction']<292.5) & (df_wt2['wind_direction']>=247.5), 'South', 
                                  'Southeast')))))))))
df_wt2['wind_direction cat2'] = np.where(df_wt2['wind_direction'].isna(), 'Unknown',
                                  np.where(df_wt2['wind_speed']==0, 'No Wind',
                                  np.where((df_wt2['wind_direction']<45)  & (df_wt2['wind_direction']>=0), 'East',
                                  np.where((df_wt2['wind_direction']<90)  & (df_wt2['wind_direction']>=45), 'Northeast',
                                  np.where((df_wt2['wind_direction']<135) & (df_wt2['wind_direction']>=90), 'North',
                                  np.where((df_wt2['wind_direction']<180) & (df_wt2['wind_direction']>=135), 'Northwest',
                                  np.where((df_wt2['wind_direction']<225) & (df_wt2['wind_direction']>=180), 'West',
                                  np.where((df_wt2['wind_direction']<270) & (df_wt2['wind_direction']>=225), 'Southwest',
                                  np.where((df_wt2['wind_direction']<315) & (df_wt2['wind_direction']>=270), 'South', 
                                  'Southeast')))))))))

df_wt2['precipitation'] = np.where(df_wt2['precip_depth_1_hr'].isna(), 'Unknown', 
                                  np.where(df_wt2['precip_depth_1_hr']<=0, 'N', 'Y'))

df_wt2['precipitation sign'] = np.where(df_wt2['precip_depth_1_hr'].isna(), 'Unknown',
                                   np.where(df_wt2['precip_depth_1_hr']<0, 'Negative',
                                   np.where(df_wt2['precip_depth_1_hr']==0, 'Zero', 'Positive')))

# df_wt2['wind_speed_log'] = np.log(df_wt2['wind_speed']+1)

#####
# weather shift
shift_hour = 1
######

cols = ['air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed', 'wind_direction cat',
       'wind_direction cat2', 'precipitation', 'precipitation sign']

for col in cols:
    df_wt2[col+'_lag'] = df_wt2[col].shift(shift_hour)
    df_wt2[col+'_lag'] = df_wt2[col].shift(shift_hour)

In [24]:
df_w2.shape

(139773, 24)

In [23]:
df_wt2.shape

(277243, 24)

In [25]:
df_w2.to_csv('data/weather_train_processed_lag.csv', index=False)
df_wt2.to_csv('data/weather_test_processed_lag.csv', index=False)

## Turn to feather

In [26]:
# train_df = pd.read_csv('Data/train.csv')
weather_train_df = pd.read_csv('Data/weather_train_processed_lag.csv')
# test_df = pd.read_csv('Data/test.csv')
weather_test_df = pd.read_csv('Data/weather_test_processed_lag.csv')
# building_meta_df = pd.read_csv('Data/building_metadata.csv')
# # sample_submission = pd.read_csv(os.path.join(root, 'sample_submission.csv'))

# train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
# test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
weather_train_df['timestamp'] = pd.to_datetime(weather_train_df['timestamp'])
weather_test_df['timestamp'] = pd.to_datetime(weather_test_df['timestamp'])

# train_df.to_feather('Data/train.feather')
# test_df.to_feather('Data/test.feather')
weather_train_df.to_feather('Data/weather_train_processed_lag.feather')
weather_test_df.to_feather('Data/weather_test_processed_lag.feather')
# building_meta_df.to_feather('Data/building_metadata.feather')
# # sample_submission.to_feather('sample_submission.feather')

## Read data and merge

In [2]:
train_df = pd.read_feather('Data/train.feather')
weather_train_df = pd.read_feather('Data/weather_train_processed_lag.feather')
test_df = pd.read_feather('Data/test.feather')
weather_test_df = pd.read_feather('Data/weather_test_processed_lag.feather')
building_meta_df = pd.read_feather('Data/building_metadata.feather')

In [3]:
train_df = train_df.merge(building_meta_df, left_on = "building_id", right_on = "building_id", how = "left")
train_df = train_df.merge(weather_train_df, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how = "left")

In [4]:
train_df.shape

(20216100, 31)

## Add some features

In [5]:
%%time
le = preprocessing.LabelEncoder()
train_df['primary_use'] = le.fit_transform(train_df['primary_use'])

move = ['meter_reading', 'timestamp'] + [x for x in train_df.columns if x not in ['meter_reading', 'timestamp']]
train_df = train_df[move]

mask =(train_df['site_id']==0) & (train_df['timestamp'] < '2016-06-01')
train_df = train_df[~mask]

# this may take mins
train_df['date_month'] = train_df['timestamp'].map(lambda x: x.month)
train_df['date_day'] = train_df['timestamp'].map(lambda x: x.day)
train_df['iso_week'] = train_df['timestamp'].map(lambda x: x.isocalendar()[1])
train_df['iso_weekend'] = train_df['timestamp'].map(lambda x: x.isocalendar()[2])
train_df['hour'] = train_df['timestamp'].map(lambda x: x.hour)
train_df['holidays'] = train_df['timestamp'].map(lambda x: 1 if x in us_holidays else 0)

Wall time: 8min 52s


In [43]:
# check = ['meter_reading',  'building_id', 'meter', 'site_id',
#        'primary_use', 
#        'air_temperature', 'cloud_coverage', 'dew_temperature',
#        'precip_depth_1_hr',  'wind_direction',
#        'wind_speed', 
#        'air_temperature_lag',
#        'cloud_coverage_lag', 'dew_temperature_lag', 'precip_depth_1_hr_lag',
#         'wind_direction_lag', 'wind_speed_lag',
#         'date_month', 'date_day',
#        'iso_week', 'iso_weekend', 'hour', 'holidays']
# train_df = train_df[check]

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19790105 entries, 0 to 19790104
Data columns (total 37 columns):
meter_reading              float64
timestamp                  datetime64[ns]
building_id                int64
meter                      int64
site_id                    int64
primary_use                int32
square_feet                int64
year_built                 float64
floor_count                float64
air_temperature            float64
cloud_coverage             float64
dew_temperature            float64
precip_depth_1_hr          float64
sea_level_pressure         float64
wind_direction             float64
wind_speed                 float64
wind_direction cat         object
wind_direction cat2        object
precipitation              object
precipitation sign         object
air_temperature_lag        float64
cloud_coverage_lag         float64
dew_temperature_lag        float64
precip_depth_1_hr_lag      float64
sea_level_pressure_lag     float64
wind_direction_la

In [10]:
train_df.columns

Index(['meter_reading', 'timestamp', 'building_id', 'meter', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed', 'wind_direction cat', 'wind_direction cat2',
       'precipitation', 'precipitation sign', 'air_temperature_lag',
       'cloud_coverage_lag', 'dew_temperature_lag', 'precip_depth_1_hr_lag',
       'sea_level_pressure_lag', 'wind_direction_lag', 'wind_speed_lag',
       'wind_direction cat_lag', 'wind_direction cat2_lag',
       'precipitation_lag', 'precipitation sign_lag', 'date_month', 'date_day',
       'iso_week', 'iso_weekend', 'hour', 'holidays'],
      dtype='object')

In [13]:
use = ['meter_reading', 'building_id', 'meter', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed', 'wind_direction cat', 'wind_direction cat2',
        'air_temperature_lag',
       'cloud_coverage_lag', 'dew_temperature_lag', 'precip_depth_1_hr_lag',
       'sea_level_pressure_lag', 'wind_direction_lag', 'wind_speed_lag',
       'wind_direction cat_lag', 'wind_direction cat2_lag',
        'date_month', 'date_day',
       'iso_week', 'iso_weekend', 'hour', 'holidays']

In [14]:
train_df = train_df[use]

In [16]:
from sklearn import preprocessing

train_df['wind_direction cat'][train_df['wind_direction cat'].isna()] = 'Unknown'
train_df['wind_direction cat2'][train_df['wind_direction cat2'].isna()] = 'Unknown'
train_df['wind_direction cat_lag'][train_df['wind_direction cat_lag'].isna()] = 'Unknown'
train_df['wind_direction cat2_lag'][train_df['wind_direction cat2_lag'].isna()] = 'Unknown'

cat_need_encode = ['wind_direction cat', 'wind_direction cat2',
                  'wind_direction cat_lag', 'wind_direction cat2_lag', 
                  ]

for var in cat_need_encode:
    print(var)
    le = preprocessing.LabelEncoder()
    train_df[var] = le.fit_transform(train_df[var])

wind_direction cat
wind_direction cat2
wind_direction cat_lag
wind_direction cat2_lag


In [17]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [18]:
train_df = reduce_mem_usage(train_df)

Mem. usage decreased to 1038.03 Mb (76.7% reduction)


In [19]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19790105 entries, 0 to 19790104
Data columns (total 32 columns):
meter_reading              float32
building_id                int16
meter                      int8
site_id                    int8
primary_use                int8
square_feet                int32
year_built                 float16
floor_count                float16
air_temperature            float16
cloud_coverage             float16
dew_temperature            float16
precip_depth_1_hr          float16
sea_level_pressure         float16
wind_direction             float16
wind_speed                 float16
wind_direction cat         int8
wind_direction cat2        int8
air_temperature_lag        float16
cloud_coverage_lag         float16
dew_temperature_lag        float16
precip_depth_1_hr_lag      float16
sea_level_pressure_lag     float16
wind_direction_lag         float16
wind_speed_lag             float16
wind_direction cat_lag     int8
wind_direction cat2_lag    int8


In [5]:
%%time
train_df = pd.read_csv('processed_automl_google.csv')

Wall time: 41.6 s


In [6]:
use = ['meter_reading', 'building_id', 'meter', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed', 'wind_direction_cat', 'wind_direction_cat2',
        'air_temperature_lag',
       'cloud_coverage_lag', 'dew_temperature_lag', 'precip_depth_1_hr_lag',
       'sea_level_pressure_lag', 'wind_direction_lag', 'wind_speed_lag',
       'wind_direction_cat_lag', 'wind_direction_cat2_lag',
        'date_month', 'date_day',
       'iso_week', 'iso_weekend', 'hour', 'holidays']

In [7]:
train_df.columns = use

In [8]:
train_df.columns

Index(['meter_reading', 'building_id', 'meter', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed',
       'wind_direction_cat', 'wind_direction_cat2', 'air_temperature_lag',
       'cloud_coverage_lag', 'dew_temperature_lag', 'precip_depth_1_hr_lag',
       'sea_level_pressure_lag', 'wind_direction_lag', 'wind_speed_lag',
       'wind_direction_cat_lag', 'wind_direction_cat2_lag', 'date_month',
       'date_day', 'iso_week', 'iso_weekend', 'hour', 'holidays'],
      dtype='object')

In [9]:
%%time
train_df = train_df.reset_index(drop=True)
train_df.to_csv('processed_automl_google.csv', index=False)

Wall time: 6min 57s
